In [32]:
import pandas as pd
import numpy as np
import camelot.io as camelot
from sklearn.linear_model import LinearRegression
import glob
import matplotlib.pyplot as plt

print(dir(camelot))
print(camelot.__file__)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

['PDFHandler', 'Path', 'StrByteType', 'Union', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'read_pdf', 'remove_extra', 'validate_input', 'warnings']
c:\Users\User\Documents\GitHub\quantitative_decision_making\.venv\Lib\site-packages\camelot\io.py


## functions

## brussels-airlines-analysis

## schiphol-analysis

In [33]:
#DEMANDE EUROP2EN POUR AMSTERDAME

# Lecture et filtrage des données
schiphol = pd.read_csv('traffic-data-schiphol.csv', sep=';', decimal=',')
schiphol = schiphol[["year","air-transport-europe", "month", "europe"]]
schiphol = schiphol[(schiphol['year'] >= 2000) & (schiphol['year'] < 2020)]

passagers_par_an = schiphol.groupby('year')[['europe']].sum()

augmentation_annuelle = passagers_par_an.pct_change() * 100
augmentation_globale = (passagers_par_an / passagers_par_an.iloc[0] - 1) * 100

n = len(passagers_par_an) - 1
valeur_debut = passagers_par_an.iloc[0]
valeur_fin = passagers_par_an.iloc[-1]
croissance_annuel = ((valeur_fin / valeur_debut) ** (1 / n) - 1) * 100

passagers_par_mois = schiphol.groupby(['year', 'month'])['europe'].sum()
total_annuel = schiphol.groupby('year')['europe'].sum()
part_mensuelle = (passagers_par_mois / total_annuel) * 100
part_mensuelle = part_mensuelle.reset_index(name='part_mensuelle_pct')
part_mensuelle_moyenne = part_mensuelle.groupby('month')['part_mensuelle_pct'].mean().reset_index()

schiphol2024 = pd.read_csv('traffic-data-schiphol.csv', sep=';', decimal=',')
schiphol2024 = schiphol2024[["year", "month", "europe"]]
schiphol2024 = schiphol2024[(schiphol2024['year'] == 2024)]
annees = [2025, 2026]
total_2024 = schiphol2024['europe'].sum()
resultats = []
for annee in annees:
    croissance_factor = (1 + croissance_annuel / 100) ** (annee - 2024)
    total_annee = total_2024 * croissance_factor
    for _, row in part_mensuelle_moyenne.iterrows():
        month = row['month']
        part_pct = row['part_mensuelle_pct']
        predicted = total_annee * (part_pct / 100)
        resultats.append([annee, month, predicted])
predictions_df = pd.DataFrame(resultats, columns=['year', 'month', 'predicted_passagers'])
print(predictions_df)




    year      month                  predicted_passagers
0   2025      april  europe   4030659.365
dtype: float64
1   2025     august  europe   4986770.856
dtype: float64
2   2025   december  europe   3319854.452
dtype: float64
3   2025   february  europe   3073816.882
dtype: float64
4   2025    january  europe   3022587.224
dtype: float64
5   2025       july  europe   5070659.305
dtype: float64
6   2025       june  europe   4536994.276
dtype: float64
7   2025      march  europe   3661245.503
dtype: float64
8   2025        may  europe   4604427.841
dtype: float64
9   2025   november  europe   3514647.045
dtype: float64
10  2025    october  europe   4558928.876
dtype: float64
11  2025  september  europe   4655153.973
dtype: float64
12  2026      april  europe   4171143.978
dtype: float64
13  2026     august  europe   5160579.782
dtype: float64
14  2026   december  europe   3435564.669
dtype: float64
15  2026   february  europe   3180951.704
dtype: float64
16  2026    january  europe   3

In [36]:
# Séparation historique / réel 2025
historique_data = brussels_data[brussels_data['year'] <= 2024]
donnees_2025_reel = brussels_data[brussels_data['year'] == 2025]

# Calcul CAGR sur historique uniquement
pass_annuel = historique_data.groupby('year')['Total number of passengers'].sum()
start_year = pass_annuel.index.min()
end_year = pass_annuel.index.max()

n = end_year - start_year
cagr = ((pass_annuel.loc[end_year] / pass_annuel.loc[start_year]) ** (1/n) - 1) * 100

# Calcul part mensuelle moyenne (sur historique uniquement)
pass_mois = historique_data.groupby(['year', 'month'])['Total number of passengers'].sum()
total_annuel = historique_data.groupby('year')['Total number of passengers'].sum()
part_mensuelle = (pass_mois / total_annuel) * 100
part_mensuelle = part_mensuelle.reset_index(name='part_mensuelle_pct')
part_mens_moyenne = part_mensuelle.groupby('month')['part_mensuelle_pct'].mean().reset_index()

# Prévisions 2025 et 2026 (sur base historique + CAGR)
total_2024 = pass_annuel.loc[2024]
total_2025 = total_2024 * (1 + cagr / 100)
total_2026 = total_2025 * (1 + cagr / 100)

pred_2025 = part_mens_moyenne.copy()
pred_2025['year'] = 2025
pred_2025['predicted_passengers'] = total_2025 * (pred_2025['part_mensuelle_pct'] / 100)

pred_2026 = part_mens_moyenne.copy()
pred_2026['year'] = 2026
pred_2026['predicted_passengers'] = total_2026 * (pred_2026['part_mensuelle_pct'] / 100)

# Fusion des prévisions uniquement (sans données réelles 2025)
previsions = pd.concat([pred_2025, pred_2026], ignore_index=True)

# Affichage final des prévisions, pas des réels
print(previsions[['year', 'month', 'predicted_passengers']])


    year      month  predicted_passengers
0   2025      April           1957165.000
1   2025     August           2415871.000
2   2025   December           1773930.000
3   2025   February           1553477.000
4   2025    January           1403192.000
5   2025       July           2450771.000
6   2025       June           2098373.000
7   2025      March           1742478.000
8   2025        May           2102553.000
9   2025   November           1756834.000
10  2025    October           2105930.000
11  2025  September           2250282.000
12  2026      April           1957165.000
13  2026     August           2415871.000
14  2026   December           1773930.000
15  2026   February           1553477.000
16  2026    January           1403192.000
17  2026       July           2450771.000
18  2026       June           2098373.000
19  2026      March           1742478.000
20  2026        May           2102553.000
21  2026   November           1756834.000
22  2026    October           2105

C:\Users\User\AppData\Local\Temp\ipykernel_21116\3811283849.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  cagr = ((pass_annuel.loc[end_year] / pass_annuel.loc[start_year]) ** (1/n) - 1) * 100


## cost-analysis

In [35]:
#fuel
aircraft_data = {'model' : ['737-800', '737-max-8'], 'gallon-hour' : [850.0, 750.0], 'cost-per-gallon' : [3.0, 3.0]}
ac_costs = pd.DataFrame(aircraft_data)
ac_costs['cost-per-hour'] = ac_costs['gallon-hour']*ac_costs['cost-per-gallon']

unit-tariff-passenger = 4.6
unit-tariff-cargo = 2.18




display(ac_costs)

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (3154940802.py, line 6)

In [ ]:
#Donnée a la main : 
Madrid : 822Miles, 2h30, entre 7 et 10 vols/j       https://www.directflights.com/BRU-MAD           	440199personnes	3042vols/ans  
Stockholm : 805Miles, 2h10, 4 (ou 3) vols/j         https://www.directflights.com/BRU-ARN
Athens : 1313Miles, 3h, 4 (ou 3) vols/j             https://www.directflights.com/BRU-ATH
Global : 202vols/j